In [ ]:
import requests
import pandas as pd

# Define the API endpoint and parameters
url = 'https://api.binance.com/api/v3/klines'
params = {
    'symbol': 'ETHUSDT',  # Replace with the desired trading pair
    'interval': '1h',     # Options: 1m, 3m, 5m, etc.
    'startTime' : 1722470400000,
    # 'endTime' : '',
    'limit': 1000
}

# Make the HTTP GET request
response = requests.get(url, params=params)
data = response.json()

# Check for errors
if response.status_code != 200:
    print(f"Error fetching data: {data['msg']}")
    exit()

# Create a DataFrame and assign column names
columns = [
    'Open time', 'Open', 'High', 'Low', 'Close', 'Volume',
    'Close time', 'Quote asset volume', 'Number of trades',
    'Taker buy base asset volume', 'Taker buy quote asset volume', 'Ignore'
]
df = pd.DataFrame(data, columns=columns)

# Convert timestamps to datetime
df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')

# Convert numeric columns to appropriate data types
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume',
                   'Quote asset volume', 'Taker buy base asset volume',
                   'Taker buy quote asset volume']
df[numeric_columns] = df[numeric_columns].astype(float)

# Display the DataFrame
print(df.head())


In [44]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np


df['Open time'] = pd.to_datetime(df['Open time'], unit='ms')
df['Close time'] = pd.to_datetime(df['Close time'], unit='ms')
numeric_columns = ['Open', 'High', 'Low', 'Close', 'Volume',
                   'Quote asset volume', 'Taker buy base asset volume',
                   'Taker buy quote asset volume']
df[numeric_columns] = df[numeric_columns].astype(float)
df = df.sort_values('Open time')


# Prepare the data
features = ['Open', 'Close', 'Volume']
data = df[features]

# Initialize separate scalers
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_y = MinMaxScaler(feature_range=(0, 1))

# Fit the scalers
scaled_data_X = scaler_X.fit_transform(data)
scaled_data_y = scaler_y.fit_transform(data[['Close']])  # Target is 'Close' price

sequence_length = 60
X = []
y = []

for i in range(sequence_length, len(scaled_data_X)):
    X.append(scaled_data_X[i - sequence_length:i])
    y.append(scaled_data_y[i, 0])  # Target is 'Close' price

X = np.array(X)
y = np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))

training_size = int(len(X) * 0.8)

X_train = X[:training_size]
y_train = y[:training_size]
X_test = X[training_size:]
y_test = y[training_size:]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


# Build the model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))

model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50))
model.add(Dropout(0.2))

model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[])


In [ ]:
# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions and actual values using scaler_y
y_pred_scaled = scaler_y.inverse_transform(y_pred)
y_test_scaled = scaler_y.inverse_transform(y_test.reshape(-1, 1))


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
#  Evaluate the model
rmse = np.sqrt(mean_squared_error(y_test_scaled, y_pred_scaled))
mae = mean_absolute_error(y_test_scaled, y_pred_scaled)

print(f"Root Mean Squared Error: {rmse}")
print(f"Mean Absolute Error: {mae}")

In [ ]:
import matplotlib.pyplot as plt

# Plot the results
plt.figure(figsize=(14,5))
plt.plot(df['Open time'][len(df) - len(y_test_scaled):], y_test_scaled, color='blue', label='Actual ETH Price')
plt.plot(df['Open time'][len(df) - len(y_test_scaled):], y_pred_scaled, color='red', label='Predicted ETH Price')
plt.title('ETH Price Prediction')
plt.xlabel('Time')
plt.ylabel('Price in USD')
plt.legend()
plt.show()